# Using IBM watsonx.governance monitoring the models deployed in Google Cloud Platform (GCP) - Vertex AI

### watsonx Credentials

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
from ibm_watson_openscale.utils.client_utils import get_my_instance_ids
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator


#masked
WOS_CREDENTIALS = {
    "url": "XXXXX",
    "username": "XXXXX",
    "apikey": "XXXXX",
}

my_instances = get_my_instance_ids(WOS_CREDENTIALS['url'], WOS_CREDENTIALS['username'], apikey = WOS_CREDENTIALS['apikey']) # Pass either password or apikey but not both
print(my_instances)

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        apikey=WOS_CREDENTIALS['apikey'],
        
        disable_ssl_verification=True
    )
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator,service_instance_id = "00000000-0000-0000-0000-000000000000")
wos_client.version


In [ ]:
# Define names
SERVICE_PROVIDER_NAME = "GCP VertexAI - All Monitors" #Custom engine name
SERVICE_PROVIDER_DESCRIPTION = "Added by VertexAI notebook to showcase monitoring Fairness, Quality, Drift and Explainability against a Custom ML provider."
SUBSCRIPTION_NAME = "Credit Default Logistic Reg Model" #Exteranl model name
CUSTOM_ML_PROVIDER_SCORING_URL = 'http://104.197.160.52:8080/predictions' #Custom engine endpoint
scoring_url = CUSTOM_ML_PROVIDER_SCORING_URL

In [ ]:
feature_columns=['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
cat_features=['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

class_label = "defaultpaymentnextmonth"
prediction_column = "prediction"
probability_columns = ['probability'] # must be an array of all probability columns.

input_parameters = {
    "label_column": class_label,
    "feature_columns": feature_columns,
    "categorical_columns": cat_features,
    "fairness_inputs": None,  
    "problem_type" : "binary"
}

In [ ]:
import pandas as pd
pd_data = pd.read_csv("UCI_Credit_Card.csv")
pd_data = pd_data.drop("ID", axis=1)
pd_data.head(10)

In [ ]:
data_df = pd_data

In [ ]:
from ibm_watson_openscale.utils.training_stats import TrainingStats

training_stats = TrainingStats(data_df,input_parameters, explain=True, fairness=False, drop_na=True)
config_json = training_stats.get_training_statistics()
config_json["notebook_version"] = 5.0

config_json

In [ ]:
# Input Schema
import copy

# remove label column from input data schema if present
input_data_schema = config_json["common_configuration"]["input_data_schema"]
input_data_schema = copy.deepcopy(input_data_schema)

is_label_column_in_input_schema = False
for field in input_data_schema["fields"]:
    if field["name"] == class_label:
        is_label_column_in_input_schema = True
        break

if is_label_column_in_input_schema:
    input_data_schema["fields"] = [x for x in input_data_schema['fields'] if x['name'] != class_label]
    
print(input_data_schema)

In [ ]:
# Training Schema

# training_data_schema = input_data_schema + class_label
training_data_schema = copy.deepcopy(input_data_schema)

schema = dict(data_df.dtypes)
class_label_dtype = str(schema[class_label])

if "object" in class_label_dtype:
    class_label_dtype = "string"
elif "int" in class_label_dtype:
    class_label_dtype = "long"
elif "float" in class_label_dtype:
    class_label_dtype = "double"

training_data_schema["fields"].append({
    "name": class_label,
    "type": class_label_dtype,
    "nullable": True,
    "metadata": {}
})

print(training_data_schema)

In [ ]:
# Output Schema

# output_data_schema = input_data_schema + prediction + probability
output_data_schema = copy.deepcopy(input_data_schema)
#output_data_schema = None
# add prediction column
# assumed datatype is same as class_label datatype
output_data_schema["fields"].append({
    "name": prediction_column,
    "type": class_label_dtype,
    "nullable": True,
    "metadata": {}
})

# add probability columns if present
# assumed datatype is "double"
if probability_columns is not None and len(probability_columns) == 1:
    output_data_schema["fields"].append({
        "name": probability_columns[0],
        "type": {
            "type": "array",
            "elementType": "double",
            "containsNull": True
        },
        "nullable": True,
        "metadata": {}
    })
elif probability_columns is not None and len(probability_columns) > 1:
    for column in probability_columns:
        output_data_schema["fields"].append({
            "name": column,
            "type": "double",
            "nullable": True,
            "metadata": {}
        })

print(output_data_schema)

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id=data_marts[0].metadata.id
print('Using existing datamart {}'.format(data_mart_id))

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
request_headers = {"Content-Type": "application/json", "Openscale": "True"}
MLCredentials = {}


CUSTOM_ENGINE_CREDENTIALS = {
    "url": scoring_url,
    "username": "admin",
    "password": "password",
}

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        request_headers=request_headers,
        operational_space_id = "production",
        credentials=CustomCredentials(
            url= CUSTOM_ENGINE_CREDENTIALS['url'],
            username= CUSTOM_ENGINE_CREDENTIALS['username'],
            password= CUSTOM_ENGINE_CREDENTIALS['password'],
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
print(wos_client.service_providers.get(service_provider_id).result)

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    if subscription.entity.asset.name == "[GCP VertexAI] " + SUBSCRIPTION_NAME:
        sub_model_id = subscription.metadata.id
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [ ]:
import uuid
asset_id = str(uuid.uuid4())
asset_name = '[GCP VertexAI] ' + SUBSCRIPTION_NAME
url = ''

asset_deployment_id = str(uuid.uuid4())
asset_deployment_name = asset_name
asset_deployment_scoring_url = scoring_url

scoring_endpoint_url = scoring_url
scoring_request_headers = {
        "Content-Type": "application/json",
        "Openscale": "True"
}

In [ ]:
# A two-step process to create subscription
# 1. Create minimal subscription without schemas
# 2. Patch subscription with Schemas

subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=asset_id,
            name=asset_name,
            url=url,
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_id,
            name=asset_deployment_name,
            deployment_type= DeploymentTypes.ONLINE,
            scoring_endpoint=ScoringEndpointRequest(
                url=scoring_endpoint_url,
                request_headers=scoring_request_headers
            )
        ),
        asset_properties=AssetPropertiesRequest(
            label_column=class_label,
            probability_fields=["probability"],
            prediction_field="predictedLabel",
            feature_fields = feature_columns,
            categorical_fields = cat_features                                                     
        )
    ).result
subscription_id = subscription_details.metadata.id

In [ ]:
subscription_id = subscription_details.metadata.id
print("Subscription id {}".format(subscription_id))

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

In [ ]:
# Patch subscription with Schemas
asset_properties = {
    "label_column": class_label,
    "prediction_field": prediction_column,
    "feature_fields": feature_columns,
    "categorical_fields": cat_features
}

if probability_columns and len(probability_columns) > 0:
    asset_properties["probability_fields"] = probability_columns
    
if input_data_schema:
    asset_properties["input_data_schema"] = input_data_schema
    
if training_data_schema:
    asset_properties["training_data_schema"] = training_data_schema
    
if output_data_schema:
    asset_properties["output_data_schema"] = output_data_schema
    
# print(asset_properties)

patch_document = [
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties', value=asset_properties)
]

response = wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=patch_document).result
# print(response)

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

In [ ]:
# get payload data set id
import time

time.sleep(20)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(
    type=DataSetTypes.PAYLOAD_LOGGING,
    target_target_id=subscription_id,
    target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id

if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

In [ ]:
# get payload record count
wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id)

In [ ]:
cols_to_remove = ['defaultpaymentnextmonth']
def get_scoring_payload(no_of_records_to_score = 1):
    for col in cols_to_remove:
        if col in pd_data.columns:
            del pd_data[col] 

    fields = pd_data.columns.tolist()
    values = pd_data[fields].values.tolist()

    payload_scoring = {"input_data": [{"fields": fields, "values": values[:no_of_records_to_score]}]}
    return payload_scoring

In [ ]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

# Put your data here
REQUEST_DATA = {"fields": ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"], "values": [[20000.0, 2.0, 2.0, 1.0, 24.0, 2.0, 2.0, -1.0, -1.0, -2.0, -2.0, 3913.0, 3102.0, 689.0, 0.0, 0.0, 0.0, 0.0, 689.0, 0.0, 0.0, 0.0, 0.0], [120000.0, 2.0, 2.0, 2.0, 26.0, -1.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2682.0, 1725.0, 2682.0, 3272.0, 3455.0, 3261.0, 0.0, 1000.0, 1000.0, 1000.0, 0.0, 2000.0]]}
RESPONSE_DATA = {"fields":["prediction","probability"],"values":[[0,[0.4703915003587882,0.5296084996412118]],[1,[0.8430531917616706,0.1569468082383294]]]}

wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(request=REQUEST_DATA, response=RESPONSE_DATA, response_time=460)])

In [ ]:
# get payload record count
wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id)

Done OpenScale subscribtion. 

Go to OpenScale UI and continoue configuration for Fairness, Drift, Explainibity and Quilty Monitioring